In [1]:
%cd C:/Users/sylee/Documents/GitHub/Final_2Kyoungh/data

C:\Users\sylee\Documents\GitHub\Final_2Kyoungh\data


In [2]:
import requests
import json
import pandas as pd

## 기본 세팅 부분
##종웅님 
headers = {"X-Naver-Client-Id": "EmdlUXsc4CuKNm_kxYKS",
           "X-Naver-Client-Secret":"dSLoIEmMgy"}

url = "https://openapi.naver.com/v1/datalab/shopping/categories"
def change_index(df, column_name):
    df = df.set_index("period")
    df.columns = [column_name]
    return df


In [3]:
devices=['','pc','mo']

## 대분류 카테고리 설정
for i in devices:
    categories = [
          {"name": "기저귀", "param": [ "50000116"]},
          {"name": "분유", "param": [ "50000115"]},
          {"name": "이유식", "param": [ "50000118"]},
          {"name": "수유용품", "param": [ "50000120"]},
          {"name": "이유식용품", "param": [ "50000132"]},
          {"name": "위생/건강용품", "param": [ "50000126"]},
          {"name": "유모차", "param": [ "50000121"]},
          {"name": "카시트", "param": [ "50000122"]},
          {"name": "외출용품", "param": [ "50000123"]},
          {"name": "유아동잡화", "param": [ "50000139"]}
      ]
## API 요청 값 작성, 디지털/가전 = result[0], 패션의류 = result[1]
    data = {
    "startDate": "2021-05-10",
    "endDate": "2023-07-03",
    "timeUnit": "date",
    "category": [
      {"name": "출산/육아", "param": [ "50000005"]},
      {"name": "완구/매트", "param": [ "50000142"]},
    ],
    "device":i,
    "gender":"",
    "ages":[]
    }
## 데이터 정제 및 merge 파트
    response = requests.post(url, headers=headers, json=data)

    results = response.json()["results"]


    df_1 = change_index(pd.DataFrame(results[0]["data"]), "출산/육아")
    df_2 = change_index(pd.DataFrame(results[1]["data"]), "완구/매트")
##
    df_all_i = pd.merge(df_1, df_2, left_on="period", right_on="period")
## 반복문 통해서 카테고리에 설정된 값, 불러오고 저장, 합치기
    for category in categories:
        if len(data["category"]) > 2:
            data["category"].pop(2)
        data["category"].append(category)

        result = requests.post(url, headers=headers, json=data)

        results = result.json()["results"]

        df = change_index(pd.DataFrame(results[2]["data"]), category["name"])

        df_all_i = pd.merge(df_all_i, df, left_on="period", right_on="period")
    if i == "":
        df_all_1 = df_all_i
        df_all_1['유형']='PC+MO'
        df_all_1['성별']='전체'
        df_all_1['연령']='전체'
    if i == "pc":
        df_all_2 = df_all_i
        df_all_2['유형']='PC'
        df_all_2['성별']='전체'
        df_all_2['연령']='전체'
    if i == "mo":
        df_all_3 = df_all_i
        df_all_3['유형']='MO'
        df_all_3['성별']='전체'
        df_all_3['연령']='전체'
##    
df_list=[df_all_1,df_all_2,df_all_3]
df_all_a=pd.concat(df_list)
df_all_a.drop(["출산/육아"], axis=1, inplace=True)
df_all_a

,완구/매트,기저귀,분유,이유식,수유용품,이유식용품,위생/건강용품,유모차,카시트,외출용품,유아동잡화,유형,성별,연령
period,,,,,,,,,,,,,,
2021-05-10,8.81675,0.44100,0.21575,0.12722,0.55295,0.68855,1.09093,0.53735,0.40534,0.29960,6.84970,PC+MO,전체,전체
2021-05-11,8.25249,0.36630,0.21040,0.11882,0.49502,0.67864,1.03169,0.56029,0.34630,0.29209,6.49746,PC+MO,전체,전체
2021-05-12,7.68905,0.34435,0.18853,0.10548,0.46581,0.63856,1.03375,0.62379,0.32381,0.28426,6.55287,PC+MO,전체,전체
2021-05-13,7.19310,0.32749,0.18444,0.09599,0.45608,0.58374,0.96047,0.75842,0.31401,0.27990,7.00459,PC+MO,전체,전체
2021-05-14,6.65694,0.28258,0.17189,0.08223,0.39717,0.53129,0.82258,0.73438,0.31553,0.30404,6.53137,PC+MO,전체,전체
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-06-29,5.02920,0.27387,0.14485,0.14150,0.33726,0.46525,0.24748,0.46457,0.21464,0.21943,3.99842,MO,전체,전체
2023-06-30,4.60250,0.23698,0.13118,0.11211,0.30714,0.40235,0.20129,0.41153,0.20245,0.20277,3.02635,MO,전체,전체
2023-07-01,6.08298,0.25360,0.14133,0.10444,0.29700,0.35069,0.20328,0.53015,0.27562,0.19851,2.96698,MO,전체,전체


In [4]:
## 대분류 카테고리 설정
for i in devices:
    categories = [
          {"name": "기저귀", "param": [ "50000116"]},
          {"name": "분유", "param": [ "50000115"]},
          {"name": "이유식", "param": [ "50000118"]},
          {"name": "수유용품", "param": [ "50000120"]},
          {"name": "이유식용품", "param": [ "50000132"]},
          {"name": "위생/건강용품", "param": [ "50000126"]},
          {"name": "유모차", "param": [ "50000121"]},
          {"name": "카시트", "param": [ "50000122"]},
          {"name": "외출용품", "param": [ "50000123"]},
          {"name": "유아동잡화", "param": [ "50000139"]}
      ]
## API 요청 값 작성, 디지털/가전 = result[0], 패션의류 = result[1]
    data = {
    "startDate": "2021-05-10",
    "endDate": "2023-07-03",
    "timeUnit": "date",
    "category": [
      {"name": "출산/육아", "param": [ "50000005"]},
      {"name": "완구/매트", "param": [ "50000142"]},
    ],
    "device":i,
    "gender":"f",
    "ages":[]
    }
## 데이터 정제 및 merge 파트
    response = requests.post(url, headers=headers, json=data)

    results = response.json()["results"]


    df_1 = change_index(pd.DataFrame(results[0]["data"]), "출산/육아")
    df_2 = change_index(pd.DataFrame(results[1]["data"]), "완구/매트")
##
    df_all_i = pd.merge(df_1, df_2, left_on="period", right_on="period")
## 반복문 통해서 카테고리에 설정된 값, 불러오고 저장, 합치기
    for category in categories:
        if len(data["category"]) > 2:
            data["category"].pop(2)
        data["category"].append(category)

        result = requests.post(url, headers=headers, json=data)

        results = result.json()["results"]

        df = change_index(pd.DataFrame(results[2]["data"]), category["name"])

        df_all_i = pd.merge(df_all_i, df, left_on="period", right_on="period")
    if i == "":
        df_all_1 = df_all_i
        df_all_1['유형']='PC+MO'
        df_all_1['성별']='여성'
        df_all_1['연령']='전체'
    if i == "pc":
        df_all_2 = df_all_i
        df_all_2['유형']='PC'
        df_all_2['성별']='여성'
        df_all_2['연령']='전체'
    if i == "mo":
        df_all_3 = df_all_i
        df_all_3['유형']='MO'
        df_all_3['성별']='여성'
        df_all_3['연령']='전체'
##    
df_list=[df_all_1,df_all_2,df_all_3]
df_all_b=pd.concat(df_list)
df_all_b.drop(["출산/육아"], axis=1, inplace=True)
df_all_b

,완구/매트,기저귀,분유,이유식,수유용품,이유식용품,위생/건강용품,유모차,카시트,외출용품,유아동잡화,유형,성별,연령
period,,,,,,,,,,,,,,
2021-05-10,10.26551,0.64453,0.31732,0.21497,0.92525,1.11541,1.71047,0.80922,0.46052,0.50465,11.40096,PC+MO,여성,전체
2021-05-11,9.62238,0.52874,0.31708,0.19905,0.82935,1.08677,1.62175,0.86823,0.39744,0.49266,10.83459,PC+MO,여성,전체
2021-05-12,8.93944,0.48414,0.28704,0.17537,0.78881,1.02814,1.65284,0.97701,0.37349,0.48149,11.02297,PC+MO,여성,전체
2021-05-13,8.37333,0.46667,0.27535,0.15656,0.73692,0.92580,1.52991,1.21474,0.35462,0.44579,11.84209,PC+MO,여성,전체
2021-05-14,7.55130,0.39907,0.26044,0.13573,0.65412,0.80920,1.29982,1.17251,0.37229,0.43214,11.10420,PC+MO,여성,전체
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-06-29,6.50023,0.41740,0.21466,0.24889,0.56531,0.82241,0.37248,0.70793,0.28262,0.36611,6.84182,MO,여성,전체
2023-06-30,5.90255,0.36505,0.19409,0.19267,0.52328,0.71611,0.30959,0.60668,0.27324,0.33530,5.15362,MO,여성,전체
2023-07-01,7.08207,0.37127,0.19919,0.17709,0.48831,0.60524,0.30419,0.74521,0.34076,0.31128,4.86580,MO,여성,전체


In [6]:
## 대분류 카테고리 설정
for i in devices:
    categories = [
          {"name": "기저귀", "param": [ "50000116"]},
          {"name": "분유", "param": [ "50000115"]},
          {"name": "이유식", "param": [ "50000118"]},
          {"name": "수유용품", "param": [ "50000120"]},
          {"name": "이유식용품", "param": [ "50000132"]},
          {"name": "위생/건강용품", "param": [ "50000126"]},
          {"name": "유모차", "param": [ "50000121"]},
          {"name": "카시트", "param": [ "50000122"]},
          {"name": "외출용품", "param": [ "50000123"]},
          {"name": "유아동잡화", "param": [ "50000139"]}
      ]
## API 요청 값 작성, 디지털/가전 = result[0], 패션의류 = result[1]
    data = {
    "startDate": "2021-05-10",
    "endDate": "2023-07-03",
    "timeUnit": "date",
    "category": [
      {"name": "출산/육아", "param": [ "50000005"]},
      {"name": "완구/매트", "param": [ "50000142"]},
    ],
    "device":i,
    "gender":"m",
    "ages":[]
    }
## 데이터 정제 및 merge 파트
    response = requests.post(url, headers=headers, json=data)

    results = response.json()["results"]


    df_1 = change_index(pd.DataFrame(results[0]["data"]), "출산/육아")
    df_2 = change_index(pd.DataFrame(results[1]["data"]), "완구/매트")
##
    df_all_i = pd.merge(df_1, df_2, left_on="period", right_on="period")
## 반복문 통해서 카테고리에 설정된 값, 불러오고 저장, 합치기
    for category in categories:
        if len(data["category"]) > 2:
            data["category"].pop(2)
        data["category"].append(category)

        result = requests.post(url, headers=headers, json=data)

        results = result.json()["results"]

        df = change_index(pd.DataFrame(results[2]["data"]), category["name"])

        df_all_i = pd.merge(df_all_i, df, left_on="period", right_on="period")
    if i == "":
        df_all_1 = df_all_i
        df_all_1['유형']='PC+MO'
        df_all_1['성별']='남성'
        df_all_1['연령']='전체'
    if i == "pc":
        df_all_2 = df_all_i
        df_all_2['유형']='PC'
        df_all_2['성별']='남성'
        df_all_2['연령']='전체'
    if i == "mo":
        df_all_3 = df_all_i
        df_all_3['유형']='MO'
        df_all_3['성별']='남성'
        df_all_3['연령']='전체'
##    
df_list=[df_all_1,df_all_2,df_all_3]
df_all_c=pd.concat(df_list)
df_all_c.drop(["출산/육아"], axis=1, inplace=True)
df_all_c

,완구/매트,기저귀,분유,이유식,수유용품,이유식용품,위생/건강용품,유모차,카시트,외출용품,유아동잡화,유형,성별,연령
period,,,,,,,,,,,,,,
2021-05-10,18.89455,0.59710,0.37612,0.10827,0.53618,0.44443,1.22319,0.81722,1.09210,0.27105,5.80026,PC+MO,남성,전체
2021-05-11,16.99865,0.46638,0.33764,0.09113,0.49080,0.41792,1.08211,0.75852,0.88665,0.25156,5.31870,PC+MO,남성,전체
2021-05-12,15.60590,0.45553,0.29522,0.07929,0.42088,0.39178,1.05979,0.80502,0.82314,0.23652,4.97070,PC+MO,남성,전체
2021-05-13,14.33930,0.41595,0.29867,0.08052,0.43210,0.35268,0.96644,0.92365,0.83178,0.22641,5.25951,PC+MO,남성,전체
2021-05-14,13.62294,0.38055,0.27018,0.06387,0.39252,0.27906,0.80909,0.91760,0.78898,0.21531,4.72764,PC+MO,남성,전체
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-06-29,11.60288,0.41420,0.26536,0.10432,0.37423,0.30105,0.36554,0.66568,0.56745,0.18387,3.50736,MO,남성,전체
2023-06-30,10.80990,0.35853,0.24512,0.09952,0.29456,0.23590,0.27263,0.61767,0.50685,0.18711,2.73877,MO,남성,전체
2023-07-01,16.91927,0.49089,0.30961,0.09913,0.34945,0.28677,0.33543,1.03706,0.81659,0.24784,3.31920,MO,남성,전체


In [7]:
df_list=[df_all_a,df_all_b,df_all_c]
df_all=pd.concat(df_list)

## 엑셀에 저장
df_all.to_excel("MCLS_08_all_fm_all.xlsx", sheet_name="click ratio")
df_all.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7065 entries, 2021-05-10 to 2023-07-03
Data columns (total 14 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   완구/매트    7065 non-null   float64
 1   기저귀      7065 non-null   float64
 2   분유       7065 non-null   float64
 3   이유식      7065 non-null   float64
 4   수유용품     7065 non-null   float64
 5   이유식용품    7065 non-null   float64
 6   위생/건강용품  7065 non-null   float64
 7   유모차      7065 non-null   float64
 8   카시트      7065 non-null   float64
 9   외출용품     7065 non-null   float64
 10  유아동잡화    7065 non-null   float64
 11  유형       7065 non-null   object 
 12  성별       7065 non-null   object 
 13  연령       7065 non-null   object 
dtypes: float64(11), object(3)
memory usage: 827.9+ KB
